In [24]:
from dotenv import load_dotenv

# API KEY 정보로드
load_dotenv()


True

In [25]:
from langchain_openai import ChatOpenAI
from langchain_teddynote import logging
from langchain_teddynote.messages import stream_response
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import FewShotChatMessagePromptTemplate
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_community.vectorstores.utils import DistanceStrategy
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_text_splitters import CharacterTextSplitter
from langchain_community.document_loaders import TextLoader
from langchain_community.document_loaders import UnstructuredMarkdownLoader
from langchain_community.document_loaders import UnstructuredExcelLoader
import pandas
from llama_parse import LlamaParse
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import re
import joblib
from langchain.schema import Document

In [26]:
# load and text split

loader = TextLoader("./db/files/2025_1_졸업이수요건_19_29.md", encoding = "utf-8")
data = loader.load()
splitter = CharacterTextSplitter(
    separator = "//<>",
    # chunk_size = 4500,
    # chunk_overlap = 500,
    # length_function = len
)


documents = splitter.split_documents(data)
len(documents)

Created a chunk of size 4224, which is longer than the specified 4000
Created a chunk of size 4700, which is longer than the specified 4000


6

In [ ]:
# save embeddings in vectorstore

embed_model = HuggingFaceEmbeddings(
    model_name = "jhgan/ko-sroberta-nli",
    model_kwargs = {"device":"cpu"},
    encode_kwargs = {"normalize_embeddings":True}
)


vectorstore = FAISS.from_documents(documents,
                                   embedding = embed_model,
                                   distance_strategy = DistanceStrategy.COSINE)


vectorstore.save_local("./db/faiss")

In [28]:
#Embedding for Keyword RAG

clean_list = [re.sub(r'[^\w\s]', ' ', str(i)) for i in documents]
clean_documents = [Document(page_content = j) for j in clean_list]

joblib.dump(clean_documents, "keyword.joblib")

['keyword.joblib']